# llc4320 drifters spectra: Lagrangian vs Eulerian

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd
from xhistogram.xarray import histogram
import xskillscore as xs
%matplotlib inline
from matplotlib import pyplot as plt
from cmocean import cm
import matplotlib.colors as colors
from matplotlib import gridspec

import mitequinox.utils as ut
import mitequinox.parcels as pa
import mitequinox.drifters as dr

INFO: Compiled ParcelsRandom ==> /dev/shm/pbs.9687164.datarmor0/parcels-502027/libparcels_random_250b225f-8303-4579-bcb7-f4fec6e1f4f5.so


In [2]:
if True:
    from dask.distributed import Client, LocalCluster
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster()
    cluster = PBSCluster(cores=28, processes=14) # necessary?
    w = cluster.scale(jobs=1)
    client = Client(cluster)
    client
else:
    from dask.distributed import Client, LocalCluster
    #
    cluster = LocalCluster()
    client = Client(cluster)
    client

In [3]:
client

Client Scheduler: tcp://10.148.0.38:40599 Dashboard: http://10.148.0.38:8787/status,Cluster Workers: 14 Cores: 28 Memory: 111.72 GiB


______________
## lat*lon maps

## (1) 30 day

In [5]:
E = '/home/datawork-lops-osi/equinox/mit4320/diags'
E_Eulerian = xr.open_zarr(os.path.join(ut.root_data_dir,'diags/SSUV_T30_nomean_lat_lon_bands.zarr'))
E_Eulerian = (E_Eulerian.rename({'E_SSUV':'E_Eulerian'}))
E_Eulerian

<xarray.Dataset>
Dimensions:         (frequency_band: 15, lat_bins: 119, lon_bins: 359)
Coordinates:
  * frequency_band  (frequency_band) object 'total' 'low' ... 'inertial_0dot4'
  * lat_bins        (lat_bins) float64 -59.5 -58.5 -57.5 ... 56.5 57.5 58.5
  * lon_bins        (lon_bins) float64 -179.5 -178.5 -177.5 ... 177.5 178.5
Data variables:
    E_Eulerian      (frequency_band, lon_bins, lat_bins) float64 dask.array<chunksize=(4, 180, 60), meta=np.ndarray>

In [7]:
root_dir = '/home/datawork-lops-osi/equinox/mit4320/parcels/'
run_name = 'global_T365j_dt1j_dij50'

p = pa.parcels_output(root_dir+run_name, parquets=['trajectory'])
E_lagrangian = p.load_diagnostic('spectra_uv_T30_mean1_lat_lon_bands.zarr')
E_lagrangian = E_lagrangian.rename({'E':'E_Lagrangian'})
E_lagrangian = E_lagrangian.E_Lagrangian
E_lagrangian

<xarray.DataArray 'E_Lagrangian' (frequency_band: 15, lon_bins: 359, lat_bins: 119)>
dask.array<open_dataset-a53f95aec3b16b40b25cbfb67e966c0bE, shape=(15, 359, 119), dtype=float64, chunksize=(1, 359, 119), chunktype=numpy.ndarray>
Coordinates:
  * frequency_band  (frequency_band) object 'total' 'low' ... 'inertial_0dot4'
  * lat_bins        (lat_bins) float64 -59.5 -58.5 -57.5 ... 56.5 57.5 58.5
  * lon_bins        (lon_bins) float64 -179.5 -178.5 -177.5 ... 177.5 178.5

In [8]:
ds_E = xr.merge([E_Eulerian,E_lagrangian])
ds_E.load()

<xarray.Dataset>
Dimensions:         (frequency_band: 15, lat_bins: 119, lon_bins: 359)
Coordinates:
  * frequency_band  (frequency_band) object 'total' 'low' ... 'inertial_0dot4'
  * lat_bins        (lat_bins) float64 -59.5 -58.5 -57.5 ... 56.5 57.5 58.5
  * lon_bins        (lon_bins) float64 -179.5 -178.5 -177.5 ... 177.5 178.5
Data variables:
    E_Eulerian      (frequency_band, lon_bins, lat_bins) float64 dask.array<chunksize=(4, 180, 60), meta=np.ndarray>
    E_Lagrangian    (frequency_band, lon_bins, lat_bins) float64 dask.array<chunksize=(1, 359, 119), meta=np.ndarray>

In [10]:
out_dir_zarr = '/home1/datawork/xyu/others/' # could change to datawork file

file_out = out_dir_zarr+'/KE_Eul_Lag_30day.nc'
%time ds_E.to_netcdf(file_out, mode='w')

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 151 ms


## (2) 60 day

In [11]:
E = '/home/datawork-lops-osi/equinox/mit4320/diags'
E_Eulerian = xr.open_zarr(os.path.join(ut.root_data_dir,'diags/SSUV_T60_nomean_lat_lon_bands.zarr'))
E_Eulerian = (E_Eulerian.rename({'E_SSUV':'E_Eulerian'}))
E_Eulerian

<xarray.Dataset>
Dimensions:         (frequency_band: 15, lat_bins: 119, lon_bins: 359)
Coordinates:
  * frequency_band  (frequency_band) object 'total' 'low' ... 'inertial_0dot4'
  * lat_bins        (lat_bins) float64 -59.5 -58.5 -57.5 ... 56.5 57.5 58.5
  * lon_bins        (lon_bins) float64 -179.5 -178.5 -177.5 ... 177.5 178.5
Data variables:
    E_Eulerian      (frequency_band, lon_bins, lat_bins) float64 dask.array<chunksize=(4, 180, 60), meta=np.ndarray>

In [12]:
root_dir = '/home/datawork-lops-osi/equinox/mit4320/parcels/'
run_name = 'global_T365j_dt1j_dij50'

p = pa.parcels_output(root_dir+run_name, parquets=['trajectory'])
E_lagrangian = p.load_diagnostic('spectra_uv_T60_mean1_lat_lon_bands.zarr')
E_lagrangian = E_lagrangian.rename({'E':'E_Lagrangian'})
E_lagrangian = E_lagrangian.E_Lagrangian
E_lagrangian

<xarray.DataArray 'E_Lagrangian' (frequency_band: 15, lon_bins: 359, lat_bins: 119)>
dask.array<open_dataset-682c1c82ba4da651f437998905b25f53E, shape=(15, 359, 119), dtype=float64, chunksize=(1, 359, 119), chunktype=numpy.ndarray>
Coordinates:
  * frequency_band  (frequency_band) object 'total' 'low' ... 'inertial_0dot4'
  * lat_bins        (lat_bins) float64 -59.5 -58.5 -57.5 ... 56.5 57.5 58.5
  * lon_bins        (lon_bins) float64 -179.5 -178.5 -177.5 ... 177.5 178.5

In [13]:
ds_E = xr.merge([E_Eulerian,E_lagrangian])
ds_E.load()

<xarray.Dataset>
Dimensions:         (frequency_band: 15, lat_bins: 119, lon_bins: 359)
Coordinates:
  * frequency_band  (frequency_band) object 'total' 'low' ... 'inertial_0dot4'
  * lat_bins        (lat_bins) float64 -59.5 -58.5 -57.5 ... 56.5 57.5 58.5
  * lon_bins        (lon_bins) float64 -179.5 -178.5 -177.5 ... 177.5 178.5
Data variables:
    E_Eulerian      (frequency_band, lon_bins, lat_bins) float64 0.04303 ... ...
    E_Lagrangian    (frequency_band, lon_bins, lat_bins) float64 0.04549 ... ...

In [14]:
out_dir_zarr = '/home1/datawork/xyu/others/' # could change to datawork file

file_out = out_dir_zarr+'/KE_Eul_Lag_60day.nc'
%time ds_E.to_netcdf(file_out, mode='w')

CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 62.6 ms


## (3) 90 day

In [15]:
E = '/home/datawork-lops-osi/equinox/mit4320/diags'
E_Eulerian = xr.open_zarr(os.path.join(ut.root_data_dir,'diags/SSUV_T90_nomean_lat_lon_bands.zarr'))
E_Eulerian = (E_Eulerian.rename({'E_SSUV':'E_Eulerian'}))
E_Eulerian

<xarray.Dataset>
Dimensions:         (frequency_band: 15, lat_bins: 119, lon_bins: 359)
Coordinates:
  * frequency_band  (frequency_band) object 'total' 'low' ... 'inertial_0dot4'
  * lat_bins        (lat_bins) float64 -59.5 -58.5 -57.5 ... 56.5 57.5 58.5
  * lon_bins        (lon_bins) float64 -179.5 -178.5 -177.5 ... 177.5 178.5
Data variables:
    E_Eulerian      (frequency_band, lon_bins, lat_bins) float64 dask.array<chunksize=(4, 180, 60), meta=np.ndarray>

In [16]:
root_dir = '/home/datawork-lops-osi/equinox/mit4320/parcels/'
run_name = 'global_T365j_dt1j_dij50'

p = pa.parcels_output(root_dir+run_name, parquets=['trajectory'])
E_lagrangian = p.load_diagnostic('spectra_uv_T90_mean1_lat_lon_bands.zarr')
E_lagrangian = E_lagrangian.rename({'E':'E_Lagrangian'})
E_lagrangian = E_lagrangian.E_Lagrangian
E_lagrangian

<xarray.DataArray 'E_Lagrangian' (frequency_band: 15, lon_bins: 359, lat_bins: 119)>
dask.array<open_dataset-e89195d19c0714f133bc3d1b067c4b05E, shape=(15, 359, 119), dtype=float64, chunksize=(1, 359, 119), chunktype=numpy.ndarray>
Coordinates:
  * frequency_band  (frequency_band) object 'total' 'low' ... 'inertial_0dot4'
  * lat_bins        (lat_bins) float64 -59.5 -58.5 -57.5 ... 56.5 57.5 58.5
  * lon_bins        (lon_bins) float64 -179.5 -178.5 -177.5 ... 177.5 178.5

In [17]:
ds_E = xr.merge([E_Eulerian,E_lagrangian])
ds_E.load()

<xarray.Dataset>
Dimensions:         (frequency_band: 15, lat_bins: 119, lon_bins: 359)
Coordinates:
  * frequency_band  (frequency_band) object 'total' 'low' ... 'inertial_0dot4'
  * lat_bins        (lat_bins) float64 -59.5 -58.5 -57.5 ... 56.5 57.5 58.5
  * lon_bins        (lon_bins) float64 -179.5 -178.5 -177.5 ... 177.5 178.5
Data variables:
    E_Eulerian      (frequency_band, lon_bins, lat_bins) float64 0.04139 ... ...
    E_Lagrangian    (frequency_band, lon_bins, lat_bins) float64 0.04929 ... ...

In [18]:
out_dir_zarr = '/home1/datawork/xyu/others/' # could change to datawork file

file_out = out_dir_zarr+'/KE_Eul_Lag_90day.nc'
%time ds_E.to_netcdf(file_out, mode='w')

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 65.4 ms


In [21]:
cluster.close()